![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/healthcare-nlp/01.2.Resume_MedicalNer_Model_Training.ipynb)

If you are using the `spark-nlp-jsl` library, please use this  [1.4.Resume_MedicalNer_Model_Training
](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/1.4.Resume_MedicalNer_Model_Training.ipynb) notebook.

# Resume MedicalNer Model Training

Steps:
- Train a new model for a few epochs.
- Load the same model and train for more epochs on the same taxnonomy, and check stats.
- Train a model already trained on a different data

## Healthcare NLP for Data Scientists Course

If you are not familiar with the components in this notebook, you can check [Healthcare NLP for Data Scientists Udemy Course](https://www.udemy.com/course/healthcare-nlp-for-data-scientists/) and the [MOOC Notebooks](https://github.com/JohnSnowLabs/spark-nlp-workshop/tree/master/Spark_NLP_Udemy_MOOC/Healthcare_NLP) for each components.

## Colab Setup

In [ ]:
# Install the johnsnowlabs library to access Spark-OCR and Spark-NLP for Healthcare, Finance, and Legal.
! pip install -q johnsnowlabs

In [ ]:
from google.colab import files
print('Please Upload your John Snow Labs License using the button below')
license_keys = files.upload()

In [3]:
from johnsnowlabs import nlp, medical, visual

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
nlp.settings.enforce_versions=True
nlp.install(refresh_install=True)

👌 Detected license file /content/6.1.1.spark_nlp_for_healthcare.json
📋 Stored John Snow Labs License in /root/.johnsnowlabs/licenses/license_number_0_for_Spark-Healthcare_Spark-OCR.json
👷 Setting up  John Snow Labs home in /root/.johnsnowlabs, this might take a few minutes.
🙆 JSL Home setup in /root/.johnsnowlabs
👌 Detected license file /content/6.1.1.spark_nlp_for_healthcare.json
Installing /root/.johnsnowlabs/py_installs/spark_nlp_jsl-6.1.1-py3-none-any.whl to /usr/bin/python3
Installed 1 products:
💊 Spark-Healthcare==6.1.1 installed! ✅ Heal the planet with NLP! 


In [4]:
from johnsnowlabs import nlp, medical, visual
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Automatically load license data and start a session with all jars user has access to
spark = nlp.start()

👌 Detected license file /content/6.1.1.spark_nlp_for_healthcare.json
👌 Launched cpu optimized session with with: 🚀Spark-NLP==6.1.3, 💊Spark-Healthcare==6.1.1, running on ⚡ PySpark==3.4.0


In [5]:
spark

## Download Clinical Word Embeddings for training

In [6]:
clinical_embeddings = nlp.WordEmbeddingsModel.pretrained('embeddings_clinical', "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]


## Download Data for Training (NCBI Disease Dataset)

In [7]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/NCBI_disease_official_test.conll
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/NCBI_disease_official_train_dev.conll

In [8]:
# from sparknlp.training import CoNLL

training_data = nlp.CoNLL().readDataset(spark, 'NCBI_disease_official_train_dev.conll')

training_data.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Identification of...|[{document, 0, 89...|[{document, 0, 89...|[{token, 0, 13, I...|[{pos, 0, 13, NN,...|[{named_entity, 0...|
|The adenomatous p...|[{document, 0, 21...|[{document, 0, 21...|[{token, 0, 2, Th...|[{pos, 0, 2, NN, ...|[{named_entity, 0...|
|Complex formation...|[{document, 0, 63...|[{document, 0, 63...|[{token, 0, 6, Co...|[{pos, 0, 6, NN, ...|[{named_entity, 0...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows



In [9]:
test_data = nlp.CoNLL().readDataset(spark, 'NCBI_disease_official_test.conll')

test_data.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Clustering of mis...|[{document, 0, 10...|[{document, 0, 10...|[{token, 0, 9, Cl...|[{pos, 0, 9, NN, ...|[{named_entity, 0...|
|Ataxia - telangie...|[{document, 0, 13...|[{document, 0, 13...|[{token, 0, 5, At...|[{pos, 0, 5, NN, ...|[{named_entity, 0...|
|The risk of cance...|[{document, 0, 15...|[{document, 0, 15...|[{token, 0, 2, Th...|[{pos, 0, 2, NN, ...|[{named_entity, 0...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows



## Split the test data into two parts:
- We Keep the first part separate and use it for training the model further, as it will be totally unseen data from the same taxonomy.

- The second part will be used to testing and evaluating

In [10]:
(test_data_1, test_data_2) = test_data.randomSplit([0.5, 0.5], seed = 100)

# save the test data as parquet for easy testing
clinical_embeddings.transform(test_data_1).write.parquet('test_1.parquet')

clinical_embeddings.transform(test_data_2).write.parquet('test_2.parquet')

## Train a new model, pause, and resume training on the same dataset.

### MedicalNerDLGraphChecker

The MedicalNerDLGraphChecker processes the dataset to extract required graph parameters (tokens, labels, embedding dimensions)

In [11]:
nerDLGraphChecker = medical.NerDLGraphChecker()\
    .setInputCols(["sentence", "token"])\
    .setLabelColumn("label")\
    .setEmbeddingsModel(clinical_embeddings)

### Train for 2 epochs

In [12]:
nerTagger = medical.NerApproach()\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setLabelColumn("label")\
    .setOutputCol("ner")\
    .setMaxEpochs(2)\
    .setLr(0.003)\
    .setBatchSize(8)\
    .setRandomSeed(0)\
    .setVerbose(1)\
    .setEvaluationLogExtended(True) \
    .setEnableOutputLogs(True)\
    .setIncludeConfidence(True)\
    .setTestDataset('./test_2.parquet')\
    .setOutputLogsPath('./ner_logs')

ner_pipeline = nlp.Pipeline(stages=[
    clinical_embeddings,
    nerDLGraphChecker,
    nerTagger
 ])

In [13]:
%%time
ner_model = ner_pipeline.fit(training_data)

CPU times: user 86.9 ms, sys: 25.2 ms, total: 112 ms
Wall time: 4min 24s


In [14]:
!ls ner_logs/MedicalNerApproach*

ner_logs/MedicalNerApproach_0af32ade2017.log


In [15]:
# Training Logs
! cat ner_logs/MedicalNerApproach*

Name of the selected graph: medical-ner-dl/blstm_100_200_128_100.pb
Training started - total epochs: 2 - lr: 0.003 - batch size: 8 - labels: 3 - chars: 84 - training examples: 6347


Epoch 1/2 started, lr: 0.003, dataset size: 6347


Epoch 1/2 - 109.44s - loss: 2246.1577 - avg training loss: 2.8253555 - batches: 795
Quality on test dataset: 
time to finish evaluation: 8.35s
Total test loss: 86.2902	Avg test loss: 1.3483
label	 tp	 fp	 fn	 prec	 rec	 f1
I-Disease	 466	 56	 164	 0.8927203	 0.73968256	 0.8090278
B-Disease	 425	 73	 106	 0.85341364	 0.80037665	 0.82604474
tp: 891 fp: 129 fn: 270 labels: 2
Macro-average	 prec: 0.87306696, rec: 0.7700296, f1: 0.8183176
Micro-average	 prec: 0.87352943, rec: 0.76744187, f1: 0.81705636


Epoch 2/2 started, lr: 0.0029850747, dataset size: 6347


Epoch 2/2 - 104.55s - loss: 812.97424 - avg training loss: 1.0226091 - batches: 795
Quality on test dataset: 
time to finish evaluation: 5.88s
Total test loss: 103.1185	Avg test loss: 1.6112
label	 tp	 f

In [16]:
# Logs of 4 consecutive epochs to compare with 2+2 epochs on separate datasets from same taxonomy

#!cat ner_logs/MedicalNerApproach_4d3d69967c3f.log

### Evaluate

In [17]:
#from sparknlp_jsl.eval import NerDLMetrics
import pyspark.sql.functions as F

pred_df = ner_model.stages[2].transform(clinical_embeddings.transform(test_data_2))

evaler = medical.NerDLMetrics(mode="full_chunk")

eval_result = evaler.computeMetricsFromDF(pred_df.select("label","ner"), prediction_col="ner", label_col="label", drop_o = True, case_sensitive = True).cache()

eval_result.withColumn("precision", F.round(eval_result["precision"],4))\
           .withColumn("recall", F.round(eval_result["recall"],4))\
           .withColumn("f1", F.round(eval_result["f1"],4)).show(100)

print(eval_result.selectExpr("avg(f1) as macro").show())
print (eval_result.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+-------+-----+----+-----+-----+---------+------+------+
| entity|   tp|  fp|   fn|total|precision|recall|    f1|
+-------+-----+----+-----+-----+---------+------+------+
|Disease|360.0|64.0|167.0|527.0|   0.8491|0.6831|0.7571|
+-------+-----+----+-----+-----+---------+------+------+

+------------------+
|             macro|
+------------------+
|0.7570977917981072|
+------------------+

None
+------------------+
|             micro|
+------------------+
|0.7570977917981072|
+------------------+

None


`ner_utils`: This new module is used after NER training to calculate mertic chunkbase and plot training logs.

`evaluate`: if verbose, returns overall performance, as well as performance per chunk type; otherwise, simply returns overall precision, recall, f1 scores

`loss_plot`: Plots the figure of loss vs epochs

`get_charts` : Plots the figures of metrics ( precision, recall, f1) vs epochs

```
from sparknlp_jsl.utils.ner_utils import get_charts, loss_plot, evaluate

metrics = evaluate(preds_df['prediction'].values, preds_df['ground_truth'].values)

loss_plot(f"./ner_logs/{log_file}")

get_charts('./ner_logs/'+log_files[0])
```

### Save the model to disk

In [18]:
ner_model.stages[2].write().overwrite().save('models/NCBI_NER_2_epoch')

### Train using the saved model on unseen dataset
#### We use unseen data from same taxonomy

In [19]:
nerTagger = medical.NerApproach()\
      .setInputCols(["sentence", "token", "embeddings"])\
      .setLabelColumn("label")\
      .setOutputCol("ner")\
      .setMaxEpochs(2)\
      .setLr(0.003)\
      .setBatchSize(8)\
      .setRandomSeed(0)\
      .setVerbose(1)\
      .setEvaluationLogExtended(True) \
      .setEnableOutputLogs(True)\
      .setIncludeConfidence(True)\
      .setTestDataset('/content/test_2.parquet')\
      .setOutputLogsPath('ner_logs')\
      .setPretrainedModelPath("models/NCBI_NER_2_epoch") ## load existing model

ner_pipeline = nlp.Pipeline(stages=[
      clinical_embeddings,
      nerDLGraphChecker,
      nerTagger
 ])

In [20]:
# remove the existing logs

! rm -r ./ner_logs

In [21]:
%%time
ner_model_retrained = ner_pipeline.fit(test_data_1)

CPU times: user 33.6 ms, sys: 11.9 ms, total: 45.5 ms
Wall time: 35 s


In [22]:
!ls ner_logs/MedicalNerApproach*

ner_logs/MedicalNerApproach_86270a928ad1.log


In [23]:
!cat ./ner_logs/MedicalNerApproach*

Name of the selected graph: pretrained graph
Training started - total epochs: 2 - lr: 0.003 - batch size: 8 - labels: 3 - chars: 84 - training examples: 438


Epoch 1/2 started, lr: 0.003, dataset size: 438


Epoch 1/2 - 10.14s - loss: 59.19676 - avg training loss: 1.0385396 - batches: 57
Quality on test dataset: 
time to finish evaluation: 5.86s
Total test loss: 57.7502	Avg test loss: 0.9023
label	 tp	 fp	 fn	 prec	 rec	 f1
I-Disease	 540	 82	 90	 0.8681672	 0.85714287	 0.8626199
B-Disease	 454	 62	 77	 0.87984496	 0.8549906	 0.8672397
tp: 994 fp: 144 fn: 167 labels: 2
Macro-average	 prec: 0.8740061, rec: 0.8560667, f1: 0.8649434
Micro-average	 prec: 0.8734622, rec: 0.8561585, f1: 0.8647238


Epoch 2/2 started, lr: 0.0029850747, dataset size: 438


Epoch 2/2 - 7.77s - loss: 31.752432 - avg training loss: 0.5570602 - batches: 57
Quality on test dataset: 
time to finish evaluation: 5.34s
Total test loss: 65.6525	Avg test loss: 1.0258
label	 tp	 fp	 fn	 prec	 rec	 f1
I-Disease	 540	 70	 

In [24]:
#from sparknlp_jsl.eval import NerDLMetrics
import pyspark.sql.functions as F

pred_df = ner_model_retrained.stages[2].transform(clinical_embeddings.transform(test_data_2))

evaler = medical.NerDLMetrics(mode="full_chunk")

eval_result = evaler.computeMetricsFromDF(pred_df.select("label","ner"), prediction_col="ner", label_col="label",case_sensitive = True).cache()

eval_result.withColumn("precision", F.round(eval_result["precision"],4))\
    .withColumn("recall", F.round(eval_result["recall"],4))\
    .withColumn("f1", F.round(eval_result["f1"],4)).show(100)

print(eval_result.selectExpr("avg(f1) as macro").show())
print (eval_result.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+-------+-----+----+----+-----+---------+------+------+
| entity|   tp|  fp|  fn|total|precision|recall|    f1|
+-------+-----+----+----+-----+---------+------+------+
|Disease|458.0|91.0|69.0|527.0|   0.8342|0.8691|0.8513|
+-------+-----+----+----+-----+---------+------+------+

+------------------+
|             macro|
+------------------+
|0.8513011152416357|
+------------------+

None
+------------------+
|             micro|
+------------------+
|0.8513011152416357|
+------------------+

None


### Train a Model Choosing Best Model

`useBestModel`:  This param preserves and restores the model that has achieved the best performance at the end of the training. The priority is metrics from testDataset (micro F1), metrics from validationSplit (micro F1), and if none is set it will keep track of loss during the training. <br/>

`earlyStopping`: You can stop training at the point when the perforfmance on test/validation dataset starts to degrage. Two params are used in order to use this feature: <br/>

- `earlyStoppingCriterion`: This is used set the minimal improvement of the test metric to terminate training. The metric monitored is the same as the metrics used in `useBestModel` (micro F1 when using test/validation set, loss otherwise). Default is 0 which means no early stopping is applied.

- `earlyStoppingPatience`:  The number of epoch without improvement which will be tolerated. Default is 0, which means that early stopping will occur at the first time when performance in the current epoch is no better than in the previous epoch.

In [25]:
# remove the existing logs

! rm -r ./ner_logs

In [26]:
nerTagger = medical.NerApproach()\
      .setInputCols(["sentence", "token", "embeddings"])\
      .setLabelColumn("label")\
      .setOutputCol("ner")\
      .setMaxEpochs(20)\
      .setLr(0.003)\
      .setBatchSize(8)\
      .setRandomSeed(0)\
      .setVerbose(1)\
      .setEvaluationLogExtended(True) \
      .setEnableOutputLogs(True)\
      .setIncludeConfidence(True)\
      .setTestDataset('./test_2.parquet')\
      .setOutputLogsPath('./ner_logs')\
      .setValidationSplit(0.2)\
      .setUseBestModel(True)\
      .setEarlyStoppingCriterion(0.04)\
      .setEarlyStoppingPatience(3)


ner_pipeline = nlp.Pipeline(stages=[
      clinical_embeddings,
      nerDLGraphChecker,
      nerTagger
 ])

In [27]:
%%time
ner_model = ner_pipeline.fit(training_data)

CPU times: user 121 ms, sys: 55.5 ms, total: 177 ms
Wall time: 12min 12s


In [28]:
!ls ner_logs/MedicalNerApproach*

ner_logs/MedicalNerApproach_6fa92f5713b9.log


In [29]:
# Training Logs
! cat ner_logs/MedicalNerApproach*

Name of the selected graph: medical-ner-dl/blstm_100_200_128_100.pb
Training started - total epochs: 20 - lr: 0.003 - batch size: 8 - labels: 3 - chars: 84 - training examples: 6347


Epoch 1/20 started, lr: 0.003, dataset size: 6347


Epoch 1/20 - 85.68s - loss: 1905.9841 - avg training loss: 2.9968305 - batches: 636
Quality on validation dataset (20.0%), validation examples = 1269
time to finish evaluation: 14.53s
Total validation loss: 269.4590	Avg validation loss: 1.6531
label	 tp	 fp	 fn	 prec	 rec	 f1
I-Disease	 1260	 304	 214	 0.8056266	 0.85481685	 0.8294931
B-Disease	 1129	 315	 121	 0.78185594	 0.9032	 0.83815885
tp: 2389 fp: 619 fn: 335 labels: 2
Macro-average	 prec: 0.7937412, rec: 0.8790084, f1: 0.8342016
Micro-average	 prec: 0.79421544, rec: 0.8770191, f1: 0.83356595
Quality on test dataset: 
time to finish evaluation: 5.53s
Total test loss: 115.1366	Avg test loss: 1.7990
label	 tp	 fp	 fn	 prec	 rec	 f1
I-Disease	 544	 168	 86	 0.76404494	 0.8634921	 0.8107303
B-Disease	

As you see above, training was stopped before 20th epoch since there were not improvement.

**Evaluate**

In [30]:
#from sparknlp_jsl.eval import NerDLMetrics

pred_df = ner_model.stages[2].transform(clinical_embeddings.transform(test_data_1))

evaler = medical.NerDLMetrics(mode="full_chunk")

eval_result = evaler.computeMetricsFromDF(pred_df.select("label","ner"), prediction_col="ner", label_col="label", drop_o = True, case_sensitive = True).cache()

eval_result.withColumn("precision", F.round(eval_result["precision"],4))\
    .withColumn("recall", F.round(eval_result["recall"],4))\
    .withColumn("f1", F.round(eval_result["f1"],4)).show(100)

print(eval_result.selectExpr("avg(f1) as macro").show())
print (eval_result.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+-------+-----+----+----+-----+---------+------+------+
| entity|   tp|  fp|  fn|total|precision|recall|    f1|
+-------+-----+----+----+-----+---------+------+------+
|Disease|377.0|87.0|51.0|428.0|   0.8125|0.8808|0.8453|
+-------+-----+----+----+-----+---------+------+------+

+------------------+
|             macro|
+------------------+
|0.8452914798206278|
+------------------+

None
+------------------+
|             micro|
+------------------+
|0.8452914798206277|
+------------------+

None


In [31]:
pred_df = ner_model.stages[2].transform(clinical_embeddings.transform(test_data_2))

evaler = medical.NerDLMetrics(mode="full_chunk")

eval_result = evaler.computeMetricsFromDF(pred_df.select("label","ner"), prediction_col="ner", label_col="label", drop_o = True, case_sensitive = True).cache()

eval_result.withColumn("precision", F.round(eval_result["precision"],4))\
    .withColumn("recall", F.round(eval_result["recall"],4))\
    .withColumn("f1", F.round(eval_result["f1"],4)).show(100)

print(eval_result.selectExpr("avg(f1) as macro").show())
print (eval_result.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+-------+-----+-----+----+-----+---------+------+------+
| entity|   tp|   fp|  fn|total|precision|recall|    f1|
+-------+-----+-----+----+-----+---------+------+------+
|Disease|450.0|113.0|77.0|527.0|   0.7993|0.8539|0.8257|
+-------+-----+-----+----+-----+---------+------+------+

+------------------+
|             macro|
+------------------+
|0.8256880733944955|
+------------------+

None
+------------------+
|             micro|
+------------------+
|0.8256880733944955|
+------------------+

None


## Now let's take a model trained on a different dataset and train on this dataset

In [32]:
jsl_ner = medical.NerModel.pretrained('ner_jsl','en','clinical/models')

jsl_ner.getClasses()

ner_jsl download started this may take some time.
Approximate size to download 14.5 MB
[OK!]


['O',
 'B-Injury_or_Poisoning',
 'B-Direction',
 'B-Test',
 'I-Route',
 'B-Admission_Discharge',
 'B-Death_Entity',
 'I-Oxygen_Therapy',
 'B-Relationship_Status',
 'I-Drug_BrandName',
 'B-Duration',
 'I-Alcohol',
 'I-Triglycerides',
 'I-Date',
 'B-Hyperlipidemia',
 'B-Respiration',
 'I-Test',
 'B-Birth_Entity',
 'I-VS_Finding',
 'B-Age',
 'I-Vaccine_Name',
 'I-Social_History_Header',
 'B-Labour_Delivery',
 'I-Medical_Device',
 'B-Family_History_Header',
 'B-BMI',
 'I-Fetus_NewBorn',
 'I-BMI',
 'B-Temperature',
 'I-Section_Header',
 'I-Communicable_Disease',
 'I-ImagingFindings',
 'I-Psychological_Condition',
 'I-Obesity',
 'I-Sexually_Active_or_Sexual_Orientation',
 'I-Modifier',
 'B-Alcohol',
 'I-Temperature',
 'I-Vaccine',
 'I-Symptom',
 'I-Pulse',
 'B-Kidney_Disease',
 'B-Oncological',
 'I-EKG_Findings',
 'B-Medical_History_Header',
 'I-Relationship_Status',
 'B-Cerebrovascular_Disease',
 'I-Blood_Pressure',
 'I-Diabetes',
 'B-Oxygen_Therapy',
 'B-O2_Saturation',
 'B-Psychological_C

### Now train a model using this model as base

In [33]:
nerTagger = medical.NerApproach()\
      .setInputCols(["sentence", "token", "embeddings"])\
      .setLabelColumn("label")\
      .setOutputCol("ner")\
      .setMaxEpochs(2)\
      .setLr(0.003)\
      .setBatchSize(8)\
      .setRandomSeed(0)\
      .setVerbose(1)\
      .setEvaluationLogExtended(True) \
      .setEnableOutputLogs(True)\
      .setIncludeConfidence(True)\
      .setTestDataset('/content/test_2.parquet')\
      .setOutputLogsPath('ner_logs')\
      .setPretrainedModelPath("/root/cache_pretrained/ner_jsl_en_4.2.0_3.0_1666181370373")\
      .setOverrideExistingTags(True) # since the tags do not align, set this flag to true

# do hyperparameter by tuning the params above (max epoch, LR, dropout etc.) to get better results
ner_pipeline = nlp.Pipeline(stages=[
      clinical_embeddings,
      nerDLGraphChecker,
      nerTagger
 ])

In [34]:
# remove the existing logs

! rm -r ./ner_logs

In [35]:
%%time
ner_jsl_retrained = ner_pipeline.fit(training_data)


CPU times: user 65.8 ms, sys: 22.1 ms, total: 87.8 ms
Wall time: 4min 31s


In [36]:
!cat ./ner_logs/MedicalNerApproach*

Name of the selected graph: pretrained graph
Training started - total epochs: 2 - lr: 0.003 - batch size: 8 - labels: 3 - chars: 103 - training examples: 6347


Epoch 1/2 started, lr: 0.003, dataset size: 6347


Epoch 1/2 - 118.78s - loss: 1380.974 - avg training loss: 1.7370743 - batches: 795
Quality on test dataset: 
time to finish evaluation: 10.50s
Total test loss: 68.1146	Avg test loss: 1.0643
label	 tp	 fp	 fn	 prec	 rec	 f1
I-Disease	 510	 57	 120	 0.8994709	 0.8095238	 0.85213035
B-Disease	 460	 87	 71	 0.84095067	 0.86629003	 0.8534323
tp: 970 fp: 144 fn: 191 labels: 2
Macro-average	 prec: 0.87021077, rec: 0.83790696, f1: 0.8537534
Micro-average	 prec: 0.87073606, rec: 0.83548665, f1: 0.8527472


Epoch 2/2 started, lr: 0.0029850747, dataset size: 6347


Epoch 2/2 - 116.51s - loss: 674.0035 - avg training loss: 0.8478031 - batches: 795
Quality on test dataset: 
time to finish evaluation: 10.20s
Total test loss: 66.9940	Avg test loss: 1.0468
label	 tp	 fp	 fn	 prec	 rec	 f1
I-Di

In [37]:
#from sparknlp_jsl.eval import NerDLMetrics

pred_df = ner_jsl_retrained.stages[2].transform(clinical_embeddings.transform(test_data_2))

evaler = medical.NerDLMetrics(mode="full_chunk")

eval_result = evaler.computeMetricsFromDF(pred_df.select("label","ner"), prediction_col="ner", label_col="label", drop_o = True, case_sensitive = True).cache()

eval_result.withColumn("precision", F.round(eval_result["precision"],4))\
    .withColumn("recall", F.round(eval_result["recall"],4))\
    .withColumn("f1", F.round(eval_result["f1"],4)).show(100)

print(eval_result.selectExpr("avg(f1) as macro").show())
print (eval_result.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+-------+-----+----+-----+-----+---------+------+------+
| entity|   tp|  fp|   fn|total|precision|recall|    f1|
+-------+-----+----+-----+-----+---------+------+------+
|Disease|421.0|63.0|106.0|527.0|   0.8698|0.7989|0.8328|
+-------+-----+----+-----+-----+---------+------+------+

+------------------+
|             macro|
+------------------+
|0.8328387734915925|
+------------------+

None
+------------------+
|             micro|
+------------------+
|0.8328387734915925|
+------------------+

None
